In [14]:
import os
import pandas as pd
import numpy as np
import openai
import tiktoken
import PyPDF2

openai.api_key = "cf0bd49030ed4aa6a6509be1cd9d604b"
openai.api_base = "https://invuniandesai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15'
tokenizer = tiktoken.get_encoding("cl100k_base")

def process_to_txt():

    input_directory = "C:/Users/Marine/Documents/CO_uniandes/2023-2/Tesis_1/MT_papers/1990-1999/"
    files=[]
    pages=[]
    raw_text = []
    
    for filename in os.listdir(input_directory):
        # Ouvrir le fichier PDF en mode lecture binaire ('rb')
        with open(input_directory+filename, 'rb') as pdf_file:
            # Créer un objet PDFReader
            try:
                pdf_reader = PyPDF2.PdfReader(pdf_file)

                # Parcourir chaque page du PDF
                for page_num in range(len(pdf_reader.pages)):
                    # Extraire le texte de la page
                    files.append(filename.split('.pdf')[0])
                    pages.append(page_num)
                    raw_text.append(pdf_reader.pages[page_num].extract_text())
                    
            except Exception as e:
                print(pdf_file," ", e)
                
    data = {'name': files, 'page': pages, 'raw_text': raw_text}
    df = pd.DataFrame(data)
    
    df['n_tokens'] = df.raw_text.apply(lambda x: len(tokenizer.encode(x)))
    return df
                
max_tokens = 1800

# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, file_name, pages, max_tokens = max_tokens):
    
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')
    
    # Get the number of tokens for each paragraph
    n_tokens = [len(tokenizer.encode(" " + paragraph)) for paragraph in paragraphs]

    chunks = []
    file=[]
    page=[]
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for paragraph, token in zip(paragraphs, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + " ")
            file.append(file_name)
            page.append(pages)
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the paragraph to the chunk and add the number of tokens to the total
        chunk.append(paragraph + " ")
        tokens_so_far += token + 1

    return chunks, file, page

def process_to_embeddings():
    shortened = []
    files=[]
    pages=[]
    # Loop through the dataframe
    df = process_to_txt()

    for row in df.iterrows():
        
        # If the text is None, go to the next row
        if len(row[1]['raw_text'])==1:
            continue

        # If the number of tokens is greater than the max number of tokens, split the text into chunks
        if row[1]['n_tokens'] > max_tokens:
            short,file,page = split_into_many(row[1]['raw_text'], row[1]['name'], row[1]['page'])
            shortened+=short
            files+=file
            pages+=page

        # Otherwise, add the text to the list of shortened texts
        else:
            shortened.append( row[1]['raw_text'] )
            files.append(row[1]['name'])
            pages.append(row[1]['page'])
            
    data = {'title': files, 'page_number': pages, 'text': shortened}      
    df =  pd.DataFrame(data)
    df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
    
    df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002-rfmanrique')['data'][0]['embedding'])

    df.to_csv('embeddings2.csv')
    print(df.head())
    return df

if __name__ == '__main__':
    df = process_to_embeddings()

                                               title  page_number  \
0  Caine, 1991. The Effects of Music on the Selec...            0   
1  Caine, 1991. The Effects of Music on the Selec...            1   
2  Caine, 1991. The Effects of Music on the Selec...            2   
3  Caine, 1991. The Effects of Music on the Selec...            3   
4  Caine, 1991. The Effects of Music on the Selec...            4   

                                                text  n_tokens  \
0  The Effects of Music on the Selected Stress Be...       391   
1  Vol. XXVIII. No. 4, Winter, 1991 181 Regardles...       555   
2  182 Journal of Music Therapy and Bee (1983) re...       458   
3  cations for use of sounds and music are extens...       440   
4  184 Journal of Music Therapy Table 1 Group Dem...       515   

                                          embeddings  
0  [-0.009731687605381012, -0.0012755125062540174...  
1  [-0.011171781457960606, -0.0006019424763508141...  
2  [-0.0274977721273899